# scRNA-Seq analysis to trace macrophage fate

## Import and settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pyranges as pr

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()

/Users/erik.fasterius/projects/6552-murmac/6552-env/bin/python
3.6.12 | packaged by conda-forge | (default, Dec  9 2020, 00:24:39) 
[GCC Clang 11.0.0]
sys.version_info(major=3, minor=6, micro=12, releaselevel='final', serial=0)


AttributeError: module 'scanpy.logging' has no attribute 'print_header'

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sc.settings.set_figure_params(dpi=80)

In [5]:
data_dir = '../data/'
work_dir = '../results/'
h5ad_dir = work_dir + 'h5ad/'

In [6]:
%load_ext rpy2.ipython

## Introduction

There are in total 4 plates of 384 well, two from 2019 and two from 2020. Well P23 and P24 were recommendated to be negative control for the sample plates. So there are in total 1528 single cells sequenced.

The sequencing center originally mapped the reads to their reference and provided counts and rpkm values, but apparently the reference genome do not contain mitochondrial sequence, and mapping rate to MT is apparently one quality control in scRNA-Seq analysis to determine if the cell is dying. (High content of MT RNA may indicated apoptosis)

## Sample info

* D21 - means 21 days after ischemia
* GFP_plus - high pdgfrb expression
* tdtomato_plus - permenant, lineage tracing marker. Indicate macrophage, Cx3cr1 expression, but there were some cells in D21 seem to have lost Cx3cr1 expression.
* tdtomato_minus - pericytes. But there are 30% macrophages that don't express tdtomato, so there is still possibility that it could be a macrophage.

cell_identity | explanation 
------------- | -----------
D21_tdtomato_minus_GFP_plus | Pericytes, but with possibility of being macrophages
D21_tdtomato_plus_GFP_plus | Macrophages, that have both tdtomato red and Pdgfrb GFP expression. There were some indication that having both might downregulate Cx3cr1 expression
D21_tdtomato_plus_GFP_plusminus | Macrophages, tdtomato red but difficult to say if it has also Pdgfrb expression due to weak expression
Healthy_tdtomato_minus_GFP_plus | Pericytes
Healthy_tdtomato_plus_GFP_plusminus | Macrophages

One thinking is to only look at tdtomato_plus to see if it gives better clustering

In [7]:
sample_info_file = data_dir + 'sampleinfo.csv'
sample_info = pd.read_csv(sample_info_file, sep=',', header=0, index_col=False)
sample_info.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/sampleinfo.csv'

In [ ]:
sample_info[['sortdate', 'cell_identity']].apply(pd.value_counts)

In [ ]:
sample_info.shape

### Add GFP and tdtomato

In [ ]:
sample_info['GFP'] = [x.split('_')[-1] for x in sample_info['cell_identity']]
tdtomato = []
for x in sample_info['cell_identity']:
    arr = x.split('_')
    if len(arr) >1:
        tdtomato.append(arr[2])
    else:
        tdtomato.append('blank')

sample_info['tdtomato'] = tdtomato

In [ ]:
sample_info.tail()

## Mapping stats - STAR

`count_table.sh` was used to parse `*_Log.final.out` and `*_ReadsPerGene.out.tab` to get `year, plate, well, n_reads, n_mapped_reads, p_mapped_reads, n_reads_in_features, p_reads_in_features, n_multi_mapped_reads, p_multi_mapped_reads, n_short_unmapped_reads, p_short_unmapped_reads, p_other`. Combined with `2019_2020_sampleinfo_276\ and\ 278_31\ and\ 33.csv` provided by Kristel. We have a complete table about the information of each cell, and its mapping info

### `fastqc` to check read quality

`fastqc` reveals that the reads are of good per base quality. Most of them have around 20% duplicated reads, and there is some overrepresented sequences. And there is no sign of adapter sequence.
Since I don't find any recommendation in trimming the reads before mapping rna-seq reads, I'll go ahead map the reads to mouse genome without trimming

### Re-mapping with `STAR`

`STAR` v2.7.7a is used for remapping. 

Reference genome and annotation were downloaded from https://www.gencodegenes.org/mouse/release_M25.html

Nucleotide sequence of the GRCm38 primary genome assembly (chromosomes and scaffolds) `GRCm38.primary_assembly.genome.fa`
Comprehensive gene annotation on the primary assembly (chromosomes and scaffolds) sequence regions `gencode.vM25.primary_assembly.annotation.gtf`

### Read in mapping stats - STAR

In [ ]:
mapping_stats_file = data_dir + 'mapping_stats.txt'
mapping_stats = pd.read_csv(mapping_stats_file, sep='\t', header=0, index_col=False)
mapping_stats.head()

In [ ]:
stats = pd.merge(mapping_stats, sample_info, how='left', left_on=['plate', 'well'], right_on=['plate_id', 'rownames'])
cell_stats = stats.drop(['plate_id', 'rownames'], axis=1)
cell_stats = cell_stats.set_index('prefix')
cell_stats.head()

### mapping distribution
Plot total number of reads mapped from each cell, separated as uniquely mapped, multi-mapped and unmapped. There are some with very low mappings, are those negative controls?

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]
sorted_cells = cell_stats[['n_mapped_reads', 'n_multi_mapped_reads', 'n_short_unmapped_reads']].\
               sort_values(by='n_mapped_reads', ascending=True).reset_index(drop=True)
sorted_cells.plot(kind='bar', stacked=True)
frame1 = plt.gca()
frame1.axes.xaxis.set_ticks([])
frame1.axes.xaxis.set_ticklabels([])
plt.xlabel('Cell')
plt.ylabel('Read count')
plt.show()

### Handle negative controls 

In [ ]:
cell_stats.loc[cell_stats['marker'] == 'blank'][['cell_identity', 'ischemia', 'marker', 'sex']]

In [ ]:
# Correct ischemia and sex for SS2_19_276_P23 and SS2_19_276_P24
cell_stats.loc[cell_stats.index.isin(['SS2_19_276_P23', 'SS2_19_276_P24']), ['ischemia', 'sex']] = 'blank'

In [ ]:
cell_stats.loc[cell_stats['marker'] == 'blank'][['cell_identity', 'ischemia', 'marker', 'sex']].head()

## Build adata

Create an anndata.AnnData object, which stores the count matrix in `X`, gene information in `var` and cell information in `obs`. 

### X matrix from counts

Counts were generated with `featureCounts`

In [ ]:
counts_file = data_dir + 'counts.txt'

In [ ]:
adata = sc.read_csv(counts_file, delimiter=None, first_column_names=True)
adata = adata.transpose()
adata

### Build var table

var is a data frame that stores gene level information. Often indexed with gene_name

gtf file has 55487 gene

In [ ]:
gtf_file = data_dir + 'gencode.vM25.primary_assembly.annotation.gtf'
genes = pr.read_gtf(gtf_file, as_df = True)
#genes = genes.df
genes.head()

In [ ]:
genes = genes[genes['Feature'] == 'gene']

In [ ]:
genes.columns

In [ ]:
genes = genes[['Chromosome', 'Source', 'gene_id', 'gene_name']].set_index('gene_name')

In [ ]:
adata.var = genes

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

### Obs table 

obs is a data frame stores cells meta data.

In [ ]:
adata.obs = cell_stats

In [ ]:
adata.obs.columns

In [ ]:
adata.obs['cell_identity'].value_counts()

## QC

Quality metrics including the percentage of mitocondrial and ribosomal genes per cell. High proportions are indicative of poor-quality cells (Islam et al. 2014; Ilicic et al. 2016), possibly because of loss of cytoplasmic RNA from perforated cells. The reasoning is that mitochondria are larger than individual transcript molecules and less likely to escape through tears in the cell membrane.


### Calculate QC metrics

In [ ]:
# mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('mt-') 
# ribosomal genes
adata.var['ribo'] = adata.var_names.str.startswith(('Mrps','Mrpl'))
# hemoglobin genes.
adata.var['hb'] = adata.var_names.str.startswith(("Hba"))

adata.var

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt','ribo','hb'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata

In [ ]:
adata.var.head()

### Save & read data 

In [ ]:
adata.layers['raw'] = adata.X
adata.obsm['raw'] = adata.X

In [ ]:
adata.write_h5ad(h5ad_dir + 'adata.raw.h5ad')

In [ ]:
adata = sc.read_h5ad(h5ad_dir + 'adata.raw.h5ad')

### Plot QC

Look at gene counts, total counts, MT percentage, ribosomal protein percentage and hemoglobin percentage, grouped by cell identity, batch, sex. 
Besides the empty well (negative controls), there are a few other cells that are sequenced at low coverage, might also be empty. Those should be filtered. High MT is only present at a few cells, while majority of them have MT percent belo 10%, and that would also be one of the cutoff.
The batch sequenced in 2020 seems to have slightly higher depth, and slightly higher MT percent, indicating batch effect, would try integration to remove batch effort later.

Percent ribosomal protein is very low. Normally it's one of the mostly expressed genes, so is it a smartseq2 feature or is it an indication of low cell quality? 

In [ ]:
sc.settings.set_figure_params(dpi=80)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, groupby = 'cell_identity', rotation= 45)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, groupby = 'sex', rotation= 45)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, groupby = 'sortdate', rotation= 45)

Empty cells clustered at the low left corner, and the cells close to them should also be filtered

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color="cell_identity")

In [ ]:
sc.pl.scatter(adata, x='n_genes_by_counts', y='total_counts', color="cell_identity")

### Filtering

A standard approach is to filter cells with low amount of reads as well as genes that are present in at least a certain amount of cells. This will also filter those empty wells. Here we will only consider cells with at least 500 detected genes and genes need to be expressed in at least 3 cells. Please note that those values are highly dependent on the library preparation method used.

In [ ]:
adata.obs[adata.obs.sex == 'blank']

In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=3)

print(adata.n_obs, adata.n_vars)

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

It's quite normal to see malat1 and mitochondrial genes in the top expressed list.

#### Filter Mito 

Ther is not so much ribosomal proteins or hemoglobins in the data, but there are some cells with high percentage of mitochondrial reads, that could be filtered ou. Since majority of them have mt <10%, will just set the cutoff so.

In [ ]:
print("cells %d"%adata.n_obs)
adata = adata[adata.obs['pct_counts_mt'] < 10, :]
print("Remaining cells %d"%adata.n_obs)

#### Filter malat1 and mt genes

As the level of expression of mitochondrial and MALAT1 genes are judged as mainly technical, it can be wise to remove them from the dataset bofore any further analysis.

In [ ]:
print(adata.n_obs, adata.n_vars)
malat1 = adata.var_names.str.startswith('malat1')
mito_genes = adata.var_names.str.startswith('mt-')

remove = np.add(mito_genes, malat1)
keep = np.invert(remove)

adata = adata[:,keep]

print(adata.n_obs, adata.n_vars)

In [ ]:
raw = adata.layers['raw']

### Plot Filtered QC

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, groupby = 'cell_identity', rotation= 45)

### Sample sex

By looking at reads from chromosome Y and XIST (X-inactive specific transcript) expression, it's quite easy to determine per sample which sex it is. It can be a good way to detect id there has been any sample mixup, to see if the sample metadata sex agree with the computational predictions.

It seems that all the ones marked as female are probably female, but not all the ones marked male are real male mice. 

In [ ]:
chrY_genes = adata.var_names[adata.var.Chromosome == 'chrY']
chrY_genes

In [ ]:
adata.obs['percent_chrY'] = np.sum(adata[:, chrY_genes].X, axis=1) / np.sum(adata.X, axis=1) * 100

In [ ]:
adata.obs['XIST-counts'] = adata.X[:, adata.var_names.str.match('Xist')]

In [ ]:
sc.pl.scatter(adata, x='XIST-counts', y='percent_chrY')

In [ ]:
sc.pl.violin(adata, ['XIST-counts', 'percent_chrY'], jitter=0.4, groupby='sex', rotation=45)

#### Makred male, but potentially female

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10,4))
ax = sns.histplot(adata.obs[(adata.obs['XIST-counts'] > 0) & (adata.obs['sex'] == 'male')]['XIST-counts'], 
                  label='male with XIST-counts >0', bins = 100, ax=axes[0])
ax.legend()
ax = sns.histplot(adata.obs[(adata.obs['XIST-counts'] > 0) & (adata.obs['sex'] == 'female')]['XIST-counts'], 
                  label='female with XIST-counts >0', bins = 100, ax=axes[1])
ax.legend()

In [ ]:
x = adata.obs[(adata.obs['XIST-counts'] > 5) & (adata.obs['sex'] == 'male')]
sns.relplot(x='XIST-counts', y='percent_chrY', data=x);

In [ ]:
adata.obs[(adata.obs['XIST-counts'] > 10) & (adata.obs['sex'] == 'male')][['year', 'plate', 'well', 'XIST-counts', 'percent_chrY', 
                                                                           'sex', 'cell_identity', 'PDGFRB_MFI']]

In [ ]:
adata.obs['contamination'] = 0
adata.obs.loc[(adata.obs['XIST-counts'] > 10) & (adata.obs['sex'] == 'male'), 'contamination'] = 1

In [ ]:
# Change sex of plate 278 to female
#adata.obs.loc[adata.obs['plate'] == 278, 'sex'] = 'female'

In [ ]:
x = adata.obs[(adata.obs['percent_chrY'] > 0.01) & (adata.obs['sex'] == 'female')]
sns.relplot(x='XIST-counts', y='percent_chrY', data=x);

#### Marked female, but potentially male

In [ ]:
adata.obs[(adata.obs['percent_chrY'] > 0.01) & (adata.obs['sex'] == 'female')][['year', 'plate', 'well', 'XIST-counts', 'percent_chrY', 
                                                                           'sex', 'cell_identity', 'PDGFRB_MFI']]

In [ ]:
adata.obs.loc[(adata.obs['percent_chrY'] > 0.01) & (adata.obs['sex'] == 'female') & (adata.obs['XIST-counts'] == 0), 'contamination'] = 1

In [ ]:
sum(adata.obs['contamination'])

plate 278 is from one mice, and was marked as male, but apparently female according to the data. Fixed in the new meta data

### Calculate cell-cycle scores

The algorithm calculates the difference of mean expression of the given list and the mean expression of reference genes. To build the reference, the function randomly chooses a bunch of genes matching the distribution of the expression of the given list. Cell cycling scoring adds three slots in the data, a score for S phase, a score for G2M phase and the predicted cell cycle phase.

In [ ]:
cell_cycle_genes_file = data_dir + 'Macosko_cell_cycle_genes.txt'
cell_cycle_genes_df = pd.read_table(cell_cycle_genes_file)
cell_cycle_genes_df

In [ ]:
#g1s_genes = cell_cycle_genes_df['IG1.S'].dropna().to_list()
s_genes = [x.lower().capitalize() for x in cell_cycle_genes_df['S'].dropna().to_list()]
g2m_genes = [x.lower().capitalize() for x in cell_cycle_genes_df['G2.M'].dropna().to_list()]
#m_genes = cell_cycle_genes_df['M'].dropna().to_list()
#mg1_genes = cell_cycle_genes_df['M.G1'].dropna().to_list()
#cell_cycle_genes = g1s_genes + s_genes + g2m_genes + m_genes + mg1_genes
cell_cycle_genes = s_genes + g2m_genes
print(len(cell_cycle_genes))
cell_cycle_genes1 = [x for x in cell_cycle_genes if x in adata.var_names]
print(len(cell_cycle_genes1))

Before running cell cycle we have to normalize the data. In scanpy object, the data slot will be overwriteen with the normalized data. So first, save the raw data into the slot `raw`.

In [ ]:
adata.raw = adata

# Normalize to depth 400 000
sc.pp.normalize_per_cell(adata, counts_per_cell_after=4e5)
sc.pp.log1p(adata)
sc.pp.scale(adata)

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
sc.pl.violin(adata, ['S_score', 'G2M_score'], jitter=0.4, groupby='cell_identity', rotation=45)

It looks like majority cells are in G2M phase, with a few in S phase and a few in G1 phase

In [ ]:
adata.obs.phase.value_counts()

### Predicting doublets

Doublets/Multiple of cells in the same well/droplet is a common issue in scRNAseq protocols. Especially in droplet-based method within overloading of cells.
Since this was flow sorted and seqeunced in smartseq2, we expect low amount of doublets.

In [ ]:
import scrublet as scr
scrub = scr.Scrublet(adata.raw.X)
adata.obs['doublet_scores'], adata.obs['predicted_doublets'] = scrub.scrub_doublets()
scrub.plot_histogram()

sum(adata.obs['predicted_doublets'])


In [ ]:
adata.obs[adata.obs['predicted_doublets'] == True][['year', 'plate', 'well', 'XIST-counts', 'percent_chrY', 
                                                    'sex', 'cell_identity', 'PDGFRB_MFI', 'contamination']]

Many of the predicted doublets were also suspicous contamination based on sex information inconsistency. 

In [ ]:
adata.obs['doublet_info'] = adata.obs["predicted_doublets"].astype(str)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, groupby = 'doublet_info', rotation=45)

Now, let's run PCA and UMAP and plot doublet scores onto umap to check the doublet prediction.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=['doublet_scores','doublet_info','cell_identity'])

Now, we'll remove all predicted doublets from the data.

In [ ]:
# Revert back to the raw counts as the main matrix in the adata
adata = adata.raw.to_adata()
adata.layers['raw'] = raw

adata = adata[adata.obs['doublet_info'] == 'False',:]
print(adata.shape)

### Save & read data

In [ ]:
adata.obsm['raw'] = adata.X

In [ ]:
adata.write_h5ad(h5ad_dir + 'adata.filtered.h5ad')

In [ ]:
adata = sc.read_h5ad(h5ad_dir + 'adata.filtered.h5ad')

## Normalization & Dimensionality reduction

In [ ]:
# Normalize to depth 400 000
sc.pp.normalize_per_cell(adata, counts_per_cell_after=4e5)
sc.pp.log1p(adata)

# Store normalized counts in the raw slot.
# We will subset adata.X for variable genes, but want to keep all genes matrix as well.
adata.raw = adata
adata

### Variable genes

We need to define which features/genes are important in our dataset to distinguish cell types. For this, we need to find genes that are highly variable across cells, which in turn will also provide a good separation of the cell clusters.

In [ ]:
# Compute variable genes
#sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=6, min_disp=0.5)
sc.pp.highly_variable_genes(adata, flavor='cell_ranger')
print('Highly variable genes: %d' %sum(adata.var.highly_variable))

# plot variable genes
sc.pl.highly_variable_genes(adata)

# subset for variable genes in the dataset

In [ ]:
adata = adata[:, adata.var['highly_variable']]

### Z-score transformation

Since each gene has a different expression level, it means that genes with higher expression values will naturally have higher variation that will be capured by PCA. This means that we need to somehow give each gene a similar weight when performing PCA. The common practice is to center and scale each gene before performing PCA. This exact scaling is called Z-score normalization. It is very useful for PCA, clustering and plotting heatmaps.

Additionally, we can use regression to remove any unwanted sources of variation from the dataset, such as `cell cycle`, `sequencing depth`, `percent mitochondria`. This is achieved by doing a generalized linear regression using these parameters as covariates in the model. Then the residuals of the model are taken as the 'regressed data'. Although pherphas not in the best way, batch effect regression can also be done here.

In [ ]:
# regress out unwanted variables
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
# scale data, clip values exceeding sd 10.
sc.pp.scale(adata, max_value=10)

### PCA

PCA is used to denoise the data

In [ ]:
sc.tl.pca(adata, svd_solver='arpack', n_comps=50)
sc.pl.pca_variance_ratio(adata, log=True, n_pcs=50)

### UMAP

UMAP is the gold standard in dimensionality reduction.
The UMAP implemented in scanpy uses neighborhood graph as the distance matrix, so we need to first calculate the graph

In [ ]:
sc.pp.neighbors(adata, n_pcs=40, n_neighbors=20)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='cell_identity')

## Integration

### Detect variable genes

Variable genes can be detected across the full dataset, but then we have the risk of having many batch-specific genes that will drive a lot of the variation. Or we can select variable genes from each batch separately to get only celltype variation. In the previous step, we've already selected variable genes, and are stored in 

In [ ]:
adata2 = adata.raw.to_adata()

In [ ]:
var_genes_all = adata.var.highly_variable
print('Highly variable genes: %d' %sum(var_genes_all))

In [ ]:
sc.pp.highly_variable_genes(adata2, flavor='cell_ranger', batch_key='sortdate')
print('Highly variable genes intersection: %d' %sum(adata2.var.highly_variable_intersection))
print('Number of batches where gene is variable:')
print(adata2.var.highly_variable_nbatches.value_counts())

In [ ]:
var_genes_batch = adata2.var.highly_variable_nbatches >0 

In [ ]:
print('Any batch var genes: %d' %sum(var_genes_batch))
print('All data var genes: %d' %sum(var_genes_all))
print('Overlap: %d' %sum(var_genes_batch & var_genes_all))
print('Variable genes in all batches: %d' %sum(adata2.var.highly_variable_nbatches == 2))
print('Overlap batch intersection and all: %d' %sum(var_genes_all & adata2.var.highly_variable_intersection))

In [ ]:
var_select = adata2.var.highly_variable_nbatches >1
var_genes = var_select.index[var_select]
len(var_genes)

In [ ]:
batches = adata.obs['sortdate'].cat.categories.tolist()
alldata = {}
for batch in batches:
    alldata[batch] = adata2[adata2.obs['sortdate'] == batch, ]
alldata

### Scanorama

In [ ]:
import scanorama

# subset the individual dataset to the same variable genes as in MNN-correct
alldata2 = dict()
for ds in alldata.keys():
    print(ds)
    alldata2[ds] = alldata[ds][:, var_genes]
    
# convert to list of AnnData objects
adatas = list(alldata2.values())

# run scanorama.integrate
scanorama.integrate_scanpy(adatas, dimred=50)

In [ ]:
# Get all the integrated matrices
scanorama_int = [ad.obsm['X_scanorama'] for ad in adatas]

# Make into one matrix
all_s = np.concatenate(scanorama_int)
print(all_s.shape)

# add to the AnnData object
adata.obsm['Scanorama'] = all_s

In [ ]:
sc.pp.neighbors(adata, n_pcs=50, use_rep='Scanorama')
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color='sortdate', title='Scanorama UMAP')

### Write & read data

In [ ]:
adata.write_h5ad(h5ad_dir + 'adata.scanorama.h5ad')

In [ ]:
adata = sc.read_h5ad(h5ad_dir + 'adata.scanorama.h5ad')

## Clustering

### Leiden

In [ ]:
sc.tl.leiden(adata, key_added='leiden_1.0') # default resolution is 1.0
sc.tl.leiden(adata, resolution=0.6, key_added='leiden_0.6')
sc.tl.leiden(adata, resolution=0.4, key_added='leiden_0.4')
sc.tl.leiden(adata, resolution=1.4, key_added='leiden_1.4')

In [ ]:
sc.pl.umap(adata, color=['leiden_0.4', 'leiden_0.6', 'leiden_1.0', 'leiden_1.4'])

### Louvain

In [ ]:
sc.tl.louvain(adata, key_added='louvain_1.0')
sc.tl.louvain(adata, resolution=0.6, key_added='louvain_0.6')
sc.tl.louvain(adata, resolution=0.4, key_added='louvain_0.4')
sc.tl.louvain(adata, resolution=1.4, key_added='louvain_1.4')

In [ ]:
sc.pl.umap(adata, color=['louvain_0.4', 'louvain_0.6', 'louvain_1.0', 'louvain_1.4'])

In [ ]:
sc.tl.leiden(adata, restrict_to=('leiden_0.4', ['6']), resolution=0.2, key_added='leiden_0.4_6_sub')
sc.pl.umap(adata, color=['leiden_0.4_6_sub'])

In [ ]:
sc.tl.leiden(adata, restrict_to=('leiden_0.4_6_sub', ['0']), resolution=0.3, key_added='leiden_0.4_6_0_sub')
sc.pl.umap(adata, color=['leiden_0.4_6_0_sub'])

## Cell type

Cell type annotation from (Oprescu et al., 2020) was used as reference and R Seurat v3 ‘FindTransferAnchors’ and ‘TransferData’ methods were used applied for cell type prediction (Stuart et al., 2019). The marker genes used in the reference were further investigated to validate the predictions. Boundary cases where the cluster and prediction did not agree, cell types annotations were changed to follow the majority cells in the cluster. 

The details are not included, rather the final cell type assignment were loaded here.

In [ ]:
clusters_d = {'2': '3',
              '0,0': '0',
              '0,1': '1', 
              '6,0': '2',
              '6,1': '4',
              '4': '5',
              '1': '6',
              '5': '7',
              '3': '8',
              '7': '9'}
adata.obs['cluster'] = [clusters_d[cl] for cl in adata.obs['leiden_0.4_6_0_sub']]

In [ ]:
adata.uns['cluster_colors'] = np.array([   '#d62728', # 0 red
                                           '#aa40fc', # 1
                                           '#ff7f0e', # 2
                                           '#1f77b4', # 3, blue
                                           '#e377c2', # 4, pink
                                           '#b5bd61', # 5, green
                                           '#279e68', # 6, green
                                           '#17becf', # 7, light blue
                                           '#8c564b', # 8, brown
                                           '#aec7e8', # 9, light blue purpleish
                                           ])

In [ ]:
clusters_ct_d = {'0': '0',
              '1': '1', 
              '2': '2',
              '3': 'APCs',
              '4': 'T-cells',
              '5': 'Tenocytes',
              '6': 'FAPs',
              '7': 'Endothelial cells',
              '8': 'Mural cells',
              '9': 'Monocytes'}
adata.obs['cluster_ct'] = [clusters_ct_d[cl] for cl in adata.obs['cluster']]
adata.obs['cluster_ct'] = adata.obs['cluster_ct'].astype('category').cat.\
    reorder_categories(['APCs', '0', '1', '2', 'Monocytes', 'T-cells', 'Tenocytes', 
                        'FAPs',  'Mural cells', 'Endothelial cells'])

In [ ]:
adata.uns['cluster_ct_colors'] = np.array(['#1f77b4', # blue
                                           '#d62728', # 0 red
                                           '#aa40fc',
                                           '#ff7f0e',
                                           '#aec7e8', # light blue purpleish
                                           '#e377c2', # 4, pink
                                           '#b5bd61', # 5, green
                                           '#279e68', # 6, green
                                           '#8c564b', # brown
                                           '#17becf', # 7, light blue
                                           ])

In [ ]:
adata.obs['cell_type'] = adata.obs['cluster_ct']
adata.obs['cell_type'].replace({'2':'Macrophages', '0':'Macrophages', '1':'Macrophages'}, inplace=True)
adata.obs['cell_type'] = adata.obs['cell_type'].astype('category').cat.\
    reorder_categories(['APCs', 'Macrophages', 'Monocytes', 'T-cells', 'Tenocytes', 
                        'FAPs',  'Mural cells', 'Endothelial cells'])

In [ ]:
adata.uns['cell_type_colors'] = np.array(['#1f77b4', # blue
                                          '#d62728', # 0 red
                                          '#aec7e8', # light blue purpleish
                                          '#e377c2', # 4, pink
                                          '#b5bd61', # 5, green
                                          '#279e68', # 6, green
                                          '#8c564b', # brown
                                          '#17becf', # 7, light blue
                                          ])

In [ ]:
adata.uns['ischemia_colors'] = np.array(['#D56B12', '#B4B4B4'])

In [ ]:
adata.uns['tdtomato_colors'] = np.array(['#64BF73', '#E52320'])

In [ ]:
sc.pl.umap(adata, color = ['cluster', 'cluster_ct', 'cell_type', 'ischemia', 'tdtomato'])

### Save data

In [ ]:
adata

In [ ]:
adata.write_h5ad(h5ad_dir + 'adata.ct.h5ad')